In [0]:
###############################
#                             #
#        ENCODE FACTORS       #
#                             #
###############################

# performs label encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
def label_encoding(df_train, df_valid, df_test):
    
    factors = df_train.select_dtypes('object').columns
    
    lbl = LabelEncoder()

    for f in factors:        
        lbl.fit(list(df_train[f].values) + list(df_valid[f].values) + list(df_test[f].values))
        df_train[f] = lbl.transform(list(df_train[f].values))
        df_valid[f] = lbl.transform(list(df_valid[f].values))
        df_test[f]  = lbl.transform(list(df_test[f].values))

    return df_train, df_valid, df_test

def one_hot_encoding(df_train, df_valid, df_test):

    factors = df_train.select_dtypes('object').columns

    lbl = OneHotEncoder()

    for f in factors:        
        lbl.fit(list(df_train[f].values) + list(df_valid[f].values) + list(df_test[f].values))
        df_train[f] = lbl.transform(list(df_train[f].values))
        df_valid[f] = lbl.transform(list(df_valid[f].values))
        df_test[f]  = lbl.transform(list(df_test[f].values))

    return df_train, df_valid, df_test

import numpy as np
def reduce_mem_usage(df, verbose = True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
from sklearn import base
class KFoldTargetEncoderTrain(base.BaseEstimator,
                               base.TransformerMixin):
    def __init__(self,colnames,targetName,
                  n_fold=5, verbosity=True,
                  discardOriginal_col=False):
        self.colnames = colnames
        self.targetName = targetName
        self.n_fold = n_fold
        self.verbosity = verbosity
        self.discardOriginal_col = discardOriginal_col
    def fit(self, X, y=None):
        return self
    def transform(self,X):
        assert(type(self.targetName) == str)
        assert(type(self.colnames) == str)
        assert(self.colnames in X.columns)
        assert(self.targetName in X.columns)
        mean_of_target = X[self.targetName].mean()
        kf = KFold(n_splits = self.n_fold,
                   shuffle = False, random_state=2019)
        col_mean_name = self.colnames + '_' + 'Kfold_Target_Enc'
        X[col_mean_name] = np.nan
        for tr_ind, val_ind in kf.split(X):
            X_tr, X_val = X.iloc[tr_ind], X.iloc[val_ind]
            X.loc[X.index[val_ind], col_mean_name] =  X_val[self.colnames].map(X_tr.groupby(self.colnames)[self.targetName].mean())
            X[col_mean_name].fillna(mean_of_target, inplace = True)
        if self.verbosity:
            encoded_feature = X[col_mean_name].values
            print('Correlation between the new feature, {} and, {} is {}.'.format(col_mean_name,self.targetName, np.corrcoef(X[self.targetName].values, encoded_feature)[0][1]))
        if self.discardOriginal_col:
            X = X.drop(self.targetName, axis=1)
        return X

class KFoldTargetEncoderTest(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,train,colNames,encodedName):
        
        self.train = train
        self.colNames = colNames
        self.encodedName = encodedName
        
    def fit(self, X, y=None):
        return self
    def transform(self,X):
        mean =  self.train[[self.colNames,
                self.encodedName]].groupby(
                                self.colNames).mean().reset_index() 
        
        dd = {}
        for index, row in mean.iterrows():
            dd[row[self.colNames]] = row[self.encodedName]
        X[self.encodedName] = X[self.colNames]
        X = X.replace({self.encodedName: dd})
        return X

In [0]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir('drive/My Drive/Colab Notebooks/WIDS/WIDS/')

FileNotFoundError: ignored

In [0]:

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
%matplotlib inline

import os
import time
import datetime
import random
import multiprocessing
import pickle

import scipy.stats

import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')


from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import log_loss, roc_auc_score, confusion_matrix

import lightgbm as lgb

In [0]:
import sys
sys.path.append('drive/My Drive/Colab Notebooks/WIDS/WIDS/codes')

In [0]:
############ RANDOMNESS

# seed function
def seed_everything(seed = 42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
# set seed
seed = 42
seed_everything(seed)

### IMPORT

In [0]:
############ DATA IMPORT

# id data
train = pd.read_csv('./raw/training_v2.csv')
test  = pd.read_csv('./raw/unlabeled.csv')


# check dimensions
print(train.shape)
print(train.shape)

train = train[-train['hospital_death'].isnull()]

(91713, 186)
(91713, 186)


In [0]:
train['NAs'] = train.isnull().sum(axis=1)
test['NAs']  = test.isnull().sum(axis=1)

In [0]:
train['hospital_id'] = train['hospital_id'].astype('object')
test['hospital_id']  = test['hospital_id'].astype('object')

train['icu_id'] = train['icu_id'].astype('object')
test['icu_id']  = test['icu_id'].astype('object')

In [0]:
for feature in train.select_dtypes('object').columns:    
    targetc = KFoldTargetEncoderTrain(feature,'hospital_death',n_fold=10)
    train = targetc.fit_transform(train)

    test_targetc = KFoldTargetEncoderTest(train,
                                          feature,
                                          f'{feature}_Kfold_Target_Enc')
    test = test_targetc.fit_transform(test)

Correlation between the new feature, hospital_id_Kfold_Target_Enc and, hospital_death is 0.061815698066015896.
Correlation between the new feature, ethnicity_Kfold_Target_Enc and, hospital_death is -0.021778031915991588.
Correlation between the new feature, gender_Kfold_Target_Enc and, hospital_death is -0.02454474750657809.
Correlation between the new feature, hospital_admit_source_Kfold_Target_Enc and, hospital_death is 0.09601692398142708.
Correlation between the new feature, icu_admit_source_Kfold_Target_Enc and, hospital_death is 0.10751265094405518.
Correlation between the new feature, icu_id_Kfold_Target_Enc and, hospital_death is 0.07331137852646111.
Correlation between the new feature, icu_stay_type_Kfold_Target_Enc and, hospital_death is -9.430062769421791e-05.
Correlation between the new feature, icu_type_Kfold_Target_Enc and, hospital_death is 0.0382226407880874.
Correlation between the new feature, apache_3j_bodysystem_Kfold_Target_Enc and, hospital_death is 0.122919143131

In [0]:
y     = train['hospital_death']
train = train.drop('hospital_death', axis=1)

In [0]:
############ FEAUTERS

# drop bad features
excluded_feats = ['encounter_id', 'patient_id', 'readmission_status', 'hospital_id', 'icu_id']

In [0]:
train['d1_diasbp_span'] = train['d1_diasbp_max'] - train['d1_diasbp_min']
test['d1_diasbp_span']  = test['d1_diasbp_max'] - test['d1_diasbp_min']

train['d1_diasbp_invasive_span'] = train['d1_diasbp_invasive_max'] - train['d1_diasbp_invasive_min']
test['d1_diasbp_invasive_span']  = test['d1_diasbp_invasive_max'] - test['d1_diasbp_invasive_min']

train['d1_diasbp_noninvasive_span'] = train['d1_diasbp_noninvasive_max'] - train['d1_diasbp_noninvasive_min']
test['d1_diasbp_noninvasive_span']  = test['d1_diasbp_noninvasive_max'] - test['d1_diasbp_noninvasive_min']

train['d1_heartrate_span'] = train['d1_heartrate_max'] - train['d1_heartrate_min']
test['d1_heartrate_span']  = test['d1_heartrate_max'] - test['d1_heartrate_min']

In [0]:
train['apache_prob_prod'] = train['apache_4a_hospital_death_prob'] * train[ 'apache_4a_icu_death_prob']
test['apache_prob_prod'] = test['apache_4a_hospital_death_prob'] * train[ 'apache_4a_icu_death_prob']

In [0]:
features = [f for f in train.columns if f not in excluded_feats]
#features = ['apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob']
print(train[features].shape)

(91713, 196)


In [0]:
############ PARAMETERS

# cores
cores = -1
# cross-validation
num_folds = 10
shuffle   = True
seed = 999
# number of trees
max_rounds = 10000
stopping   = 200
verbose    = 250

# LGB parameters
lgb_params = {
    'boosting_type':     'gbdt',
    'objective':         'binary',
    'metric':            'auc',
    'bagging_fraction':  0.9,
    'feature_fraction':  0.9,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
    'min_split_gain':    0.1,
    'min_child_weight':  0.1,
    'min_child_samples': 10,
    'silent':            True,
    'verbosity':         -1,
    'learning_rate':     0.01,
    'max_depth':         5,
    'num_leaves':        64,
    'scale_pos_weight':  1,
    'n_estimators':      max_rounds,
    'nthread' :          cores,
    'random_state':      seed,
    #"device" : "gpu"
}

# data partitinoing
folds = StratifiedKFold(n_splits = num_folds, random_state = seed, shuffle = shuffle)
#folds = GroupKFold(n_splits = num_folds)
#folds = model_selection.TimeSeriesSplit(n_splits = 10)

# SMOTE settings
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state = seed, n_jobs = cores, sampling_strategy = 0.05)

In [0]:

############ PLACEHOLDERS

# placeholders
clfs = []
importances = pd.DataFrame()

# predictions
preds_test   = np.zeros(test.shape[0])
preds_oof    = np.zeros(train.shape[0])

In [0]:
############ CROSS-VALIDATION LOOP
cv_start  = time.time()
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, y)):

    # data partitioning
    trn_x, trn_y = train[features].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = train[features].iloc[val_idx], y.iloc[val_idx]
    test_x       = test[features]
        
    # Fill Na
    trn_x['weight']  = trn_x['weight'].fillna(trn_x.groupby(['ethnicity','age','gender'])['weight'].transform('mean'))
    val_x['weight']  = val_x['weight'].fillna(trn_x.groupby(['ethnicity','age','gender'])['weight'].transform('mean'))
    test_x['weight'] = test_x['weight'].fillna(trn_x.groupby(['ethnicity','age','gender'])['weight'].transform('mean'))
    
    trn_x['height']  = trn_x['height'].fillna(trn_x.groupby(['ethnicity','age','gender'])['height'].transform('mean'))
    val_x['height']  = val_x['height'].fillna(trn_x.groupby(['ethnicity','age','gender'])['height'].transform('mean'))
    test_x['height'] = test_x['height'].fillna(trn_x.groupby(['ethnicity','age','gender'])['height'].transform('mean'))
    
    trn_x['bmi']  = trn_x['bmi'].fillna(trn_x.groupby(['ethnicity','age','gender'])['bmi'].transform('mean'))
    val_x['bmi']  = val_x['bmi'].fillna(trn_x.groupby(['ethnicity','age','gender'])['bmi'].transform('mean'))
    test_x['bmi'] = test_x['bmi'].fillna(trn_x.groupby(['ethnicity','age','gender'])['bmi'].transform('mean'))
    
    for column in trn_x.select_dtypes('object').columns:
        trn_x[column] = trn_x[column].fillna('')
        val_x[column] = val_x[column].fillna('')
        test_x[column] = test_x[column].fillna('')
        
    # label encoding
    trn_x, val_x, test_x = label_encoding(trn_x, val_x, test_x)
    
    
    ## remove outliers
    # num_vars = trn_x.select_dtypes(include='number')
    #num_vars = num_vars.columns
    #for num_var in num_vars:
    #    trn_x[num_var] = trn_x[num_var].replace([np.inf, -np.inf], np.nan)
    #    trn_x[num_var] = trn_x[num_var].fillna(trn_x[num_var].median())
    #out_idx = (np.abs(scipy.stats.zscore(trn_x[num_vars])) < 20).all(axis = 1) + (trn_y.values == 1)
    #trn_x = trn_x[out_idx]
    
    ## scale data
    #val_x  = val_x.replace([np.inf, -np.inf], np.nan)
    
    #scaler   = RobustScaler()
    #trn_x    = pd.DataFrame(scaler.fit_transform(trn_x))
    #val_x    = pd.DataFrame(scaler.transform(val_x))
    #test_x   = pd.DataFrame(scaler.transform(test_x))
       
    ## add noise to train to reduce overfitting
    trn_x += np.random.normal(0, 0.01, trn_x.shape)
    
    # print data dimensions
    print('Data shape:', trn_x.shape, val_x.shape)
    #print('Data shape:', trn_y.shape, val_y.shape)    
    # train lightGBM
    clf = lgb.LGBMClassifier(**lgb_params) 
    clf = clf.fit(trn_x, trn_y, 
                  eval_set              = [(trn_x, trn_y), (val_x, val_y)], 
                  eval_metric           = 'auc', 
                  early_stopping_rounds = stopping,
                  verbose               = verbose)
    clfs.append(clf)
    
    # find the best iteration
    best_iter = clf.best_iteration_

    # save predictions
    preds_oof[val_idx] = clf.predict_proba(val_x,  num_iteration = best_iter)[:, 1]
    preds_test        += clf.predict_proba(test_x, num_iteration = best_iter)[:, 1] / folds.n_splits 

    # importance
    fold_importance_df               = pd.DataFrame()
    fold_importance_df['Feature']    = trn_x.columns
    fold_importance_df['Importance'] = clf.feature_importances_
    fold_importance_df['Fold']       = n_fold + 1
    importances                      = pd.concat([importances, fold_importance_df], axis = 0)
    
    # print performance
    print('--------------------------------')
    print('FOLD%2d: AUC = %.6f' % (n_fold + 1, roc_auc_score(y[val_idx], preds_oof[val_idx])))
    print('--------------------------------')
    print('')
        
    # clear memory
    del trn_x, trn_y, val_x, val_y
    gc.collect()
    
    
# print overall performance    
cv_perf = roc_auc_score(y, preds_oof)
print('--------------------------------')
print('- OOF AUC = %.6f' % cv_perf)
print('- CV TIME = {:.2f} min'.format((time.time() - cv_start) / 60))
print('--------------------------------')

Data shape: (82541, 196) (9172, 196)
Training until validation scores don't improve for 200 rounds.
[250]	training's auc: 0.909307	valid_1's auc: 0.891833
[500]	training's auc: 0.924394	valid_1's auc: 0.899632
[750]	training's auc: 0.935105	valid_1's auc: 0.902844
[1000]	training's auc: 0.943594	valid_1's auc: 0.904784
[1250]	training's auc: 0.950714	valid_1's auc: 0.905799
[1500]	training's auc: 0.956355	valid_1's auc: 0.906521
[1750]	training's auc: 0.961084	valid_1's auc: 0.906976
[2000]	training's auc: 0.965638	valid_1's auc: 0.907353
[2250]	training's auc: 0.969795	valid_1's auc: 0.907384
Early stopping, best iteration is:
[2119]	training's auc: 0.967728	valid_1's auc: 0.907512
--------------------------------
FOLD 1: AUC = 0.907512
--------------------------------

Data shape: (82541, 196) (9172, 196)
Training until validation scores don't improve for 200 rounds.
[250]	training's auc: 0.909159	valid_1's auc: 0.886305
[500]	training's auc: 0.925606	valid_1's auc: 0.893515
[750]	tr

### EVALUATION

In [0]:
############ RECHECK PERFORMANCE  

# check performance
print(np.round(roc_auc_score(y, preds_oof), 5))


############ TRACK RESULTS

In [0]:
############ VARIABLE IMPORTANCE

# load importance    
top_feats = 300
cols = importances[['Feature', 'Importance']].groupby('Feature').mean().sort_values(by = 'Importance', ascending = False)[0:top_feats].index
importance = importances.loc[importances.Feature.isin(cols)]
    
# plot variable importance
plt.figure(figsize = (10, 150))
sns.barplot(x = 'Importance', y = 'Feature', data = importance.sort_values(by = 'Importance', ascending = False))
plt.tight_layout()
plt.savefig('../var_importance.pdf')

SUBMISSION

In [0]:
# file name
model = 'lgb_v30'
perf  = str(round(cv_perf, 6))[2:7]
name  = model + '_' + perf
name

In [0]:
# export OOF preds
oof = pd.DataFrame({'encounter_id': train['encounter_id'], 'hospital_death': preds_oof})
oof.to_csv('./oof_preds/' + str(name) + '.csv', index = False)
oof.head()

In [0]:

# export submission
sub = pd.DataFrame({'encounter_id': test['encounter_id'], 'hospital_death': preds_test})
sub.to_csv('./submissions/' + str(name) + '.csv', index = False)
sub.head()